# ibm_db.prepare()

## Purpose:

Send an SQL statement to a Db2 server or database to have it prepared for execution.

## Syntax:

`IBM_DBStatement ibm_db.prepare( IBM_DBConnection `*`connection,`*` string `*`SQLstatement`*` [, dictionary `*`options`*`] )`

## Parameters:

* __*connection*__&nbsp; &nbsp; &nbsp; &nbsp; : A valid Db2 server or database connection.
* __*SQLstatement*__&nbsp; &nbsp;: A valid, executable SQL statement (that may or may not contain one or more parameter markers).
* __options__&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; : A dictionary containing key-value pairs for the statement attributes that are to be set when the statement provided in the __*SQLstatement*__ parameter is prepared. Valid keys and values are:<p>
        
    * `ibm_db.SQL_ATTR_CURSOR_TYPE`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: Specifies the type of cursor that is to be used for processing result sets. Valid values are:
        * `ibm_db.SQL_CURSOR_FORWARD_ONLY`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A forward only cursor should be used (i.e., a cursor that only scrolls forward).
        * `ibm_db. SQL_CURSOR_KEYSET_DRIVEN`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A keyset-driven cursor should be used.
        * `ibm_db.SQL_CURSOR_STATIC`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A static cursor should be used.
        * `ibm_db.SQL_CURSOR_DYNAMIC`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: A dynamic, scrollable cursor that detects all changes to the result set, should be used. Dynamic cursors are only supported by Db2 for z/OS servers.
    * `ibm_db.SQL_ATTR_ROWCOUNT_PREFETCH`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: Enables Db2 to determine the number of rows that are returned by a query (so the entire result set can be prefetched into memory, when possible); `ibm_db.SQL_ROWCOUNT_PREFETCH_ON` will turn this behavior __ON__ and `ibm_db.SQL_ROWCOUNT_PREFETCH_OFF` will turn it __OFF__.
    * `ibm_db.SQL_ATTR_QUERY_TIMEOUT`&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;: The number of seconds to wait for an SQL statement to execute before aborting and returning to the application. This option can be used to terminate long running queries &mdash; the default value of __0__ means an application will wait indefinitely for the server to complete execution.<p>

## Return values:

* If __successful__, a valid `IBM_DBStatement` object.
* If __unsuccessful__, the value `None`.

## Description:

The __ibm_db.prepare()__ API is used to send an SQL statement to a Db2 server or database to have it prepared for execution &mdash; once an SQL statement has been prepared, it can be submitted for execution multiple times without having to be re-prepared. This benefits applications in two ways:<p>

* __Improved performance__: when an SQL statement is prepared, the database server creates an optimized access plan for executing that statement. Subsequent executions of the prepared statement (using the __ibm_db.execute()__ or __ibm_db.execute_many()__ API) will reuse that access plan, eliminating the overhead of dynamically creating new access plans each time the statement is executed.<p>
    
* __Greater flexibility__: when an SQL statement is prepared, literal values hard-coded in the statement can be replaced with parameter markers &mdash; depicted by question mark (?) characters. An SQL statement can contain zero or more parameter markers, each representing a variable or value that is to be provided at run time. Values for parameter markers can be supplied via variables (using the __ibm_db.bind_param()__ API), or for input parameters only, in a tuple (if the __ibm_db.execute()__ API is used) or a tuple of tuples (if the __ibm_db.execute_many()__ API is used). Regardless of which method used, when the prepared statement is executed, the database server will check each input value supplied to ensure the appropriate data type is used.<p>
    
It is important to note that parameter markers are only allowed in certain places in an SQL statement. For example, they cannot be used to represent one or more columns in the result set reurned by a __SELECT__ statement, nor can they be used as the operand of a binary operator like the equal sign (=).

## Example:

In [1]:
#----------------------------------------------------------------------------------------------#
#  NAME:     ibm_db-prepare.py                                                                 #
#                                                                                              #
#  PURPOSE:  This program is designed to illustrate how to use the ibm_db.prepare() API.       #
#                                                                                              #
#            Additional APIs used:                                                             #
#                 ibm_db.execute()                                                             #
#                 ibm_db.fetch_assoc()                                                         #
#                                                                                              #
#----------------------------------------------------------------------------------------------#
#                     DISCLAIMER OF WARRANTIES AND LIMITATION OF LIABILITY                     #
#                                                                                              #
#  (C) COPYRIGHT International Business Machines Corp. 2018, 2019 All Rights Reserved          #
#  Licensed Materials - Property of IBM                                                        #
#                                                                                              #
#  US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA    #
#  ADP Schedule Contract with IBM Corp.                                                        #
#                                                                                              #
#  The following source code ("Sample") is owned by International Business Machines            #
#  Corporation ("IBM") or one of its subsidiaries and is copyrighted and licensed, not sold.   #
#  You may use, copy, modify, and distribute the Sample in any form without payment to IBM,    #
#  for the purpose of assisting you in the creation of Python applications using the ibm_db    #
#  library.                                                                                    #
#                                                                                              #
#  The Sample code is provided to you on an "AS IS" basis, without warranty of any kind. IBM   #
#  HEREBY EXPRESSLY DISCLAIMS ALL WARRANTIES, EITHER EXPRESS OR IMPLIED, INCLUDING, BUT NOT    #
#  LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE. #
#  Some jurisdictions do not allow for the exclusion or limitation of implied warranties, so   #
#  the above limitations or exclusions may not apply to you. IBM shall not be liable for any   #
#  damages you suffer as a result of using, copying, modifying or distributing the Sample,     #
#  even if IBM has been advised of the possibility of such damages.                            #
#----------------------------------------------------------------------------------------------#

# Load The Appropriate Python Modules
import sys         # Provides Information About Python Interpreter Constants And Functions
import ibm_db      # Contains The APIs Needed To Work With Db2 Databases

#----------------------------------------------------------------------------------------------#
# Import The Db2ConnectionMgr Class Definition, Attributes, And Methods That Have Been Defined #
# In The File Named "ibm_db_tools.py"; This Class Contains The Programming Logic Needed To     #
# Establish And Terminate A Connection To A Db2 Server Or Database                             #
#----------------------------------------------------------------------------------------------#
from ibm_db_tools import Db2ConnectionMgr

#----------------------------------------------------------------------------------------------#
# Import The ipynb_exit Class Definition, Attributes, And Methods That Have Been Defined In    #
# The File Named "ipynb_exit.py"; This Class Contains The Programming Logic Needed To Allow    #
# "exit()" Functionality To Work Without Raising An Error Or Stopping The Kernel If The        #
# Application Is Invoked In A Jupyter Notebook                                                 #
#----------------------------------------------------------------------------------------------#
from ipynb_exit import exit

# Define And Initialize The Appropriate Variables
dbName = "SAMPLE"
userID = "db2inst1"
passWord = "Passw0rd"
dbConnection = None
preparedStmt = False
returnCode = False
dataRecord = None

# Create An Instance Of The Db2ConnectionMgr Class And Use It To Connect To A Db2 Database
conn = Db2ConnectionMgr('DB', dbName, '', '', userID, passWord)
conn.openConnection()
if conn.returnCode is True:
    dbConnection = conn.connectionID
else:
    conn.closeConnection()
    exit(-1)

# Define The SQL Statement That Is To Be Executed
sqlStatement = "SELECT lastname, firstnme FROM employee WHERE job = 'PRES'"

# Prepare The SQL Statement Just Defined
print("Preparing the SQL statement \"" + sqlStatement + "\" ... ", end="")
try:
    preparedStmt = ibm_db.prepare(dbConnection, sqlStatement)
except Exception:
    pass

# If The SQL Statement Could Not Be Prepared By Db2, Display An Error Message And Exit
if preparedStmt is False:
    print("\nERROR: Unable to prepare the SQL statement specified.")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# Execute The SQL Statement Just Prepared
print("Executing the prepared SQL statement ... ", end="")
try:
    returnCode = ibm_db.execute(preparedStmt)
except Exception:
    pass

# If The SQL Statement Could Not Be Executed, Display An Error Message And Exit 
if returnCode is False:
    print("\nERROR: Unable to execute the SQL statement specified.")
    conn.closeConnection()
    exit(-1)

# Otherwise, Complete The Status Message
else:
    print("Done!\n")

# Retrieve The Data Produced By The SQL Statement And Store It In A Python Dictionary
try:
    dataRecord = ibm_db.fetch_assoc(preparedStmt)
except:
    pass

# If The Data Could Not Be Retrieved, Display An Error Message And Exit 
if returnCode is False:
    print("\nERROR: Unable to retrieve the data produced by the SQL statement.")
    conn.closeConnection()
    exit(-1)

# If The Data Could be Retrieved, Display It
else:
    print("Query results:\n")
    print("FIRSTNME    LASTNAME")
    print("__________  ________")
    print("{:10}  {:<24}\n" .format(dataRecord['FIRSTNME'], dataRecord['LASTNAME']))

# Close The Database Connection That Was Opened Earlier
conn.closeConnection()

# Return Control To The Operating System
exit()


Connecting to the SAMPLE database ... Done!

Preparing the SQL statement "SELECT lastname, firstnme FROM employee WHERE job = 'PRES'" ... Done!

Executing the prepared SQL statement ... Done!

Query results:

FIRSTNME    LASTNAME
__________  ________
CHRISTINE   HAAS                    

Disconnecting from the SAMPLE database ... Done!

